In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch import nn, optim
from torch.cuda.amp import GradScaler, autocast
import os
import numpy as np
import time

In [10]:
# Data loading and transformation setup
def load_data(data_dir):
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    full_dataset = ImageFolder(root=data_dir)
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    train_dataset.dataset.transform = transform_train
    test_dataset.dataset.transform = transform_test

    class_counts = np.bincount([full_dataset.targets[i] for i in train_dataset.indices])
    class_weights = 1. / (class_counts + 1e-6)
    sample_weights = class_weights[[full_dataset.targets[i] for i in train_dataset.indices]]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

    return train_dataset, test_dataset, sampler

In [11]:
# Model definition
def create_model(num_classes):
    model = resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.BatchNorm1d(512),
        nn.Linear(512, num_classes)
    )
    return model

In [12]:
# Model training with mixed precision
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.to(device)
    scaler = GradScaler()
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if i % 20 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Step {i}, Loss: {loss.item()}')

        elapsed_time = time.time() - start_time
        print(f'Training time for epoch {epoch+1}: {elapsed_time:.2f} seconds')


In [13]:
# Model evaluation
def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy}%')



In [14]:

# Main function
if __name__ == "__main__":
    data_dir = '/content/drive/MyDrive/JPEGImages'
    train_dataset, test_dataset, sampler = load_data(data_dir)

    train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

    model = create_model(num_classes=len(train_dataset.dataset.classes))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_model(model, train_loader, criterion, optimizer)
    evaluate_model(model, test_loader)


Epoch 1/10, Step 0, Loss: 4.14190673828125
Epoch 1/10, Step 20, Loss: 3.1257553100585938
Epoch 1/10, Step 40, Loss: 2.7007944583892822
Epoch 1/10, Step 60, Loss: 2.3544106483459473
Epoch 1/10, Step 80, Loss: 2.5450572967529297
Epoch 1/10, Step 100, Loss: 2.5978689193725586
Epoch 1/10, Step 120, Loss: 2.2019705772399902
Training time for epoch 1: 80.00 seconds
Epoch 2/10, Step 0, Loss: 1.7251014709472656
Epoch 2/10, Step 20, Loss: 1.8758130073547363
Epoch 2/10, Step 40, Loss: 1.6479625701904297
Epoch 2/10, Step 60, Loss: 1.8215947151184082
Epoch 2/10, Step 80, Loss: 1.5668410062789917
Epoch 2/10, Step 100, Loss: 1.445160150527954
Epoch 2/10, Step 120, Loss: 1.4572689533233643
Training time for epoch 2: 50.74 seconds
Epoch 3/10, Step 0, Loss: 1.58429753780365
Epoch 3/10, Step 20, Loss: 1.4516232013702393
Epoch 3/10, Step 40, Loss: 1.45829176902771
Epoch 3/10, Step 60, Loss: 1.49888014793396
Epoch 3/10, Step 80, Loss: 1.246508240699768
Epoch 3/10, Step 100, Loss: 1.0307159423828125
Epoch 